In [1]:
from os import path

PROJECT_ROOT = path.abspath(path.join(globals()['_dh'][0], '..'))
DATALAKE_PATH = path.abspath(path.join(PROJECT_ROOT, '..', '..', 'datalake', 'loraproj'))
FQFN_PROCESSED_DF = path.join(DATALAKE_PATH, 'processed', f'processed_20250215.jsonl')
MODEL_DIR = path.abspath(path.join(PROJECT_ROOT, 'loraproj', 'classifier'))

In [2]:
import pandas as pd

df = pd.read_json(f'file://{FQFN_PROCESSED_DF}', orient='records', lines=True)
print(f'df shape={df.shape}')
print(f'df columns={df.columns}')

df shape=(704, 7)
df columns=Index(['file_name', 'fqfn', 'text_body', 'input_ids', 'attention_mask',
       'token_type_ids', 'label'],
      dtype='object')


In [3]:
from loraproj.trainer.lora_trainer import LoraTrainer, TrainerConf

trainer = LoraTrainer(df, TrainerConf())

2025-02-13 19:41:10,654 - tensorcraft - INFO - XLA Device Not Supported: No module named 'torch_xla'
2025-02-13 19:41:10,663 - tensorcraft - INFO - Pytorch version=2.6.0 preferred device=mps build with MPS support=True
2025-02-13 19:41:10,668 - tensorcraft - INFO - resolved device_name: mps compute_device: mps tensor_device: mps
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 442,368 || all params: 125,089,538 || trainable%: 0.3536
LoRA model=microsoft/graphcodebert-base on mps with #trainable_parameters=None


In [4]:
from time import time
start_time = time()

trainer.train()

print(f'Training duration: {time() - start_time:.4f} seconds')

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1,Roc Auc
1,No log,0.029122,1.000000,1.000000,1.000000,1.000000,1.000000
2,No log,0.009162,1.000000,1.000000,1.000000,1.000000,1.000000
3,No log,0.007058,1.000000,1.000000,1.000000,1.000000,1.000000
4,No log,0.006661,1.000000,1.000000,1.000000,1.000000,1.000000
5,No log,0.005834,1.000000,1.000000,1.000000,1.000000,1.000000
6,No log,0.005191,1.000000,1.000000,1.000000,1.000000,1.000000
7,No log,0.004796,1.000000,1.000000,1.000000,1.000000,1.000000
8,0.060400,0.004394,1.000000,1.000000,1.000000,1.000000,1.000000
9,0.060400,0.004255,1.000000,1.000000,1.000000,1.000000,1.000000
10,0.060400,0.004107,1.000000,1.000000,1.000000,1.000000,1.000000


Training duration: 511.1791 seconds


In [7]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch

# Load the fine-tuned model
model = AutoModelForSequenceClassification.from_pretrained('./lora_graphcodebert_powershell/checkpoint-852')
tokenizer = AutoTokenizer.from_pretrained('./lora_graphcodebert_powershell/checkpoint-852')

# New PowerShell script to classify
new_script = """
function Get-SensitiveInfo {
    param($User)
    Get-ADUser -Identity $User -Properties PasswordLastSet, AccountExpirationDate
}
"""

# Tokenize input script
inputs = tokenizer(new_script, truncation=True, padding='max_length', max_length=512, return_tensors='pt')

# Predict
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

# Interpret results
logits = outputs.logits
predicted_class = torch.argmax(logits).item()

label_map = {0: 'Benign', 1: 'Malicious'}
print(f'Prediction: {label_map[predicted_class]}')


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Prediction: Benign
